<a href="https://colab.research.google.com/github/AIWintermuteAI/aXeleRate/blob/dev/resources/aXeleRate_standford_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Standford Dog Breed Classification model Training and Inference

In this notebook we will use axelerate Keras-based framework for AI on the edge to quickly setup model training and then after training session is completed convert it to .tflite and .kmodel formats.

First, let's take care of some administrative details. 

1) Before we do anything, make sure you have choosen GPU as Runtime type (in Runtime - > Change Runtime type).

2) We need to mount Google Drive for saving our model checkpoints and final converted model(s). Press on Mount Google Drive button in Files tab on your left. 

In the next cell we clone axelerate Github repository and import it. 

**It is possible to use pip install or python setup.py install, but in that case you will need to restart the enironment.** Since I'm trying to make the process as streamlined as possibile I'm using sys.path.append for import.

In [ ]:
#we need imgaug 0.4 for image augmentations to work properly, see https://stackoverflow.com/questions/62580797/in-colab-doing-image-data-augmentation-with-imgaug-is-not-working-as-intended
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4
!git clone https://github.com/AIWintermuteAI/aXeleRate.git
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference

At this step you typically need to get the dataset. You can use !wget command to download it from somewhere on the Internet or !cp to copy from My Drive as in this example
```
!cp -r /content/drive/'My Drive'/pascal_20_segmentation.zip .
!unzip --qq pascal_20_segmentation.zip
```
For this notebook we will use Standford Dog Breed Classification dataset for fine-grained classification, which you can download here:
http://vision.stanford.edu/aditya86/ImageNetDogs/

In the next cell we will download the same dataset, but with training/validation split already done - I shared on my Google Drive. We will also download pre-trained model to demonstrate inference results.

Let's visualize our classification validation dataset with visualize_dataset function, which will search for all images in folder and display num_imgs number of images with class overlayer over the image.


In [ ]:
%matplotlib inline
!gdown https://drive.google.com/uc?id=1qq758Tjsfm7Euu9ev7hSyLkMj63YC9ST  #dog breed classification dataset
!gdown https://drive.google.com/uc?id=1dFnDCOxws2uX4ZpauSPC6r6jdjHoJw_p  #pre-trained model
!unzip --qq dogs_classification.zip

from axelerate.networks.common_utils.augment import visualize_classification_dataset

visualize_classification_dataset('dogs_classification/imgs_validation', num_imgs=10, img_size=224, augment=True)

Next step is defining a config dictionary. Most lines are self-explanatory.

Type is model frontend - Classifier, Detector or Segnet

Architecture is model backend (feature extractor) 

- Full Yolo
- Tiny Yolo
- MobileNet1_0
- MobileNet7_5 
- MobileNet5_0 
- MobileNet2_5 
- SqueezeNet
- NASNetMobile
- DenseNet121
- ResNet50

**Note that while you can train any network type with any backend (Tiny YOLO + Classifier, NASNETMobile +  Detector, DenseNet121 + Segnet and so on), some converters do not support larger networks! E.g. K210 converter only supports MobileNet and TinyYOLO backends.**

Fully_connected is number of neurons in classification layers as list.

Dropout value is dropout in classification layers.

actual_epoch is number of epochs to train, noramlly good starting value is 50 - 100

train_times is a multiplier for training dataset, i.e. how many times to repeat the dataset during one epoch. Useful when you apply augmentations to image. Normally between 1 and 3 is okay. If you have big dataset, can leave at 1.

For converter type you can choose the following:

'k210', 'tflite_fullint', 'tflite_dynamic', 'edgetpu', 'openvino', 'onnx'

**Since it is an example notebook, we will use pretrained weights and set all layers of the model to be "frozen"(non-trainable), except for the last one. Also we set learning rate to very low value, that will allow us to see the perfomance of pretrained model** 

In [ ]:
config = {
    "model" : {
        "type":                 "Classifier",
        "architecture":         "NASNetMobile",
        "input_size":           224,
        "fully-connected":      [],
        "labels":               [],
        "dropout" : 		0.2
    },
     "weights" : {
            "full":   				"/content/Classifier_best_val_accuracy.h5",
            "backend":   		    "imagenet",
            "save_bottleneck":      False
        
    },
    "train" : {
        "actual_epoch":         1,
        "train_image_folder":   "dogs_classification/imgs",
        "train_times":          1,
        "valid_image_folder":   "dogs_classification/imgs_validation",
        "valid_times":          1,
        "valid_metric":         "val_accuracy",
        "batch_size":           16,
        "learning_rate":        0.0,
        "saved_folder":   		F"/content/drive/MyDrive/dogs_classifier",
        "first_trainable_layer": "dense",
        "augumentation":				True
    },
    "converter" : {
        "type":   				[]
    }
}

Let's check what GPU we have been assigned in this Colab session, if any.

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Also, let's open Tensorboard, where we will be able to watch model training progress in real time. Training and validation logs also will be saved in project folder.
Since there are no logs before we start the training, tensorboard will be empty. Refresh it after first epoch.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
!sleep 10

Finally we start the training by passing config dictionary we have defined earlier to setup_training function. The function will start the training with Checkpoint, Reduce Learning Rate on Plateu and Early Stopping callbacks. Every time our validation metric(in this config set to "val_accuracy") improves, the model is saved with Checkpoint callback. If you have specified the converter type in the config, after the training has stopped the script will convert the best model into the format you have specified in config and save it to the project folder.

In [ ]:
from keras import backend as K 
K.clear_session()
model_path = setup_training(config_dict=config)

After training it is good to check the actual perfomance of your model by doing inference on your validation dataset and visualizing results. This is exactly what next block does. Our model used pre-trained weights and since all the layers,except for the last one were set as non-trainable and we set the learning rate to a very low value, we are just observing the perfomance of the model that was trained before.

In [ ]:
%matplotlib inline
from keras import backend as K 
K.clear_session()
setup_inference(config, model_path)

If you need to convert trained model to other formats, for example for inference with Edge TPU or Kendryte K210, you can do it with following commands. Specify the converter type, backend and folder with calbiration images(normally your validation image folder).

In [ ]:
from axelerate.networks.common_utils.convert import Converter
converter = Converter('tflite_dynamic', 'NASNetMobile', 'dogs_classification/imgs_validation')
converter.convert_model(model_path)

To train the model from scratch use the following config and then run the cells with training and (optinally) inference functions again.

In [ ]:
config = {
    "model" : {
        "type":                 "Classifier",
        "architecture":         "NASNetMobile",
        "input_size":           224,
        "fully-connected":      [],
        "labels":               [],
        "dropout" : 		0.2
    },
     "weights" : {
            "full":   				"",
            "backend":   		    "imagenet",
            "save_bottleneck":      False
        
    },
    "train" : {
        "actual_epoch":         50,
        "train_image_folder":   "dogs_classification/imgs",
        "train_times":          1,
        "valid_image_folder":   "dogs_classification/imgs_validation",
        "valid_times":          1,
        "valid_metric":         "val_accuracy",
        "batch_size":           16,
        "learning_rate":        1e-3,
        "saved_folder":   		F"/content/drive/MyDrive/dogs_classifier",
        "first_trainable_layer": "",
        "augumentation":				True
    },
    "converter" : {
        "type":   				["tflite_dynamic"]
    }
}

In [ ]:
from keras import backend as K 
K.clear_session()
model_path = setup_training(config_dict=config)

In [ ]:
%matplotlib inline
from keras import backend as K 
K.clear_session()
setup_inference(config, model_path)

Good luck and happy training! Have a look at these articles, that would allow you to get the most of Google Colab or connect to local runtime if there are no GPUs available;

https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403

https://research.google.com/colaboratory/local-runtimes.html